块应用，递归检索，indexnode索引子块，代码来源：https://docs.llamaindex.ai/en/stable/examples/retrievers/recursive_retriever_nodes/
思路是：将文档分成若干个大块，为all_node,再将每个大块分成若干个小块，为all_node_sub，然后递归检索，直到all_node_sub中的每个小块都检索完毕，返回all_node的结果。
作用是：分析一个大块内容中内每个小块与问题的相似度，然后返回相似度最高的大块。比通常用大块内容与问题相似度计算返回的知识较为精确。

认为可以改进的地方：增加递归深度，可采用父节点总结子节点的内容。检索方式从根节点依次检索，每层都候选3个最优的节点，且只保存最优的3个节点，每一层都更新最后的3个节点。减少检索时间。
个人认为最好的方式为：从叶子节点开始，依次向上检索，直到根节点。候选3个最优节点。

In [1]:
from pathlib import Path
from llama_index.readers.file import PDFReader,MarkdownReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json

In [2]:
try:
    from env import *
except ModuleNotFoundError as e:
    import sys
    sys.path.append('/opt/product/v1')
    from env import *
    print(f"Warning: {e}. The 'env' module could not be found, continuing without it.")


2024-07-04 08:13:56.625 | INFO     | env:<module>:23 - qwen2:7b 语言模型已经加载成功


In [3]:
loader = MarkdownReader()
docs0 = loader.load_data(file=Path("./knowledge_data/23903-i00.md"))    # 加载pdf
print(docs0)

[Document(id_='d3307085-dc18-4898-b5d1-dec4b090c325', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\nForeword\n\r\nThis Technical Report has been produced by the 3rd Generation Partnership Project (3GPP).\r\n\r\nThe contents of the present document are subject to continuing work within the TSG and may change following formal TSG approval. Should the TSG modify the contents of the present document, it will be re-released by the TSG with an identifying change of release date and an increase in version number as follows:\r\n\r\nVersion x.y.z\r\n\r\nwhere:\r\n\r\nx\tthe first digit:\r\n\r\n1\tpresented to TSG for information;\r\n\r\n2\tpresented to TSG for approval;\r\n\r\n3\tor greater indicates TSG approved document under change control.\r\n\r\ny\tthe second digit is incremented for all changes of substance, i.e. technical enhancements, corrections, updates, etc.\r\n\r\nz\tthe third digit is incremented when editor

In [4]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0]) # 分割
docs = [Document(text=doc_text)]
print(docs)
print(docs[0])

[Document(id_='da5d27b7-5b2b-4f47-9c18-d5a7971580ae', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\nForeword\n\r\nThis Technical Report has been produced by the 3rd Generation Partnership Project (3GPP).\r\n\r\nThe contents of the present document are subject to continuing work within the TSG and may change following formal TSG approval. Should the TSG modify the contents of the present document, it will be re-released by the TSG with an identifying change of release date and an increase in version number as follows:\r\n\r\nVersion x.y.z\r\n\r\nwhere:\r\n\r\nx\tthe first digit:\r\n\r\n1\tpresented to TSG for information;\r\n\r\n2\tpresented to TSG for approval;\r\n\r\n3\tor greater indicates TSG approved document under change control.\r\n\r\ny\tthe second digit is incremented for all changes of substance, i.e. technical enhancements, corrections, updates, etc.\r\n\r\nz\tthe third digit is incremented when editor

In [5]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

In [6]:
node_parser = SentenceSplitter(chunk_size=1024)

In [7]:
base_nodes = node_parser.get_nodes_from_documents(docs)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [8]:
from llama_index.core.embeddings import resolve_embed_model
from env import *
embed_model = resolve_embed_model("local:bge-small-en-v1_5")

/usr/local/anaconda3/envs/cosmetic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [18]:
all_nodes_dict = {n.node_id: n for n in all_nodes}  # 结构  key:indexnode
def custom_serializer(obj):
    if isinstance(obj, IndexNode):
        return obj.to_dict()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# 将dict保存成json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(all_nodes_dict, f, ensure_ascii=False, indent=4,default=custom_serializer)

In [11]:
vector_index_chunk = VectorStoreIndex(all_nodes, embed_model=embed_model)

In [12]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=3)

In [13]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [34]:
question = "In a place where there is no signal from Operator A but there is a signal from Operator B, will users of Operator A connect to Operator B's network?"
nodes = retriever_chunk.retrieve(
    question
)
print(str(nodes))
for node_ in nodes:
    print(str(node_))

Retrieving with query id None: In a place where there is no signal from Operator A but there is a signal from Operator B, will users of Operator A connect to Operator B's network?
Retrieved node with id, entering: node-4
Retrieving with query id node-4: In a place where there is no signal from Operator A but there is a signal from Operator B, will users of Operator A connect to Operator B's network?
Retrieved node with id, entering: node-5
Retrieving with query id node-5: In a place where there is no signal from Operator A but there is a signal from Operator B, will users of Operator A connect to Operator B's network?
Retrieved node with id, entering: node-7
Retrieving with query id node-7: In a place where there is no signal from Operator A but there is a signal from Operator B, will users of Operator A connect to Operator B's network?
[NodeWithScore(node=IndexNode(id_='node-4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships=

In [32]:
response = llm.invoke([SystemMessage(content="You are a helpful assistant."), HumanMessage(content=f"Reference content:{str(nodes)},text represents content related to the problem, and the relevant score is Score,Answer questions by combining reference content with personal knowledge：question:{question}")])

Yes, in such a scenario, when a user (UE) of Operator A loses coverage within the UTRAN network and moves into an area with coverage provided by Operator B's GERAN network, the procedure described can be used for fallback from UTRAN Video to GERAN Voice call. This means that UE 2 will connect to Operator B's network.

However, there are complications in this process:

a) If the UTRAN cell is attached to a different MSC (Mobile Switching Center) and the GERAN cell is attached to another MSC, problems might occur due to the time required for UE 2 to perform Location Area and Routeing Area updating. This can lead to issues like UE 2 not responding to MSC 2's paging.

b) In order to avoid unnecessary handover attempts, RNCs (Radio Network Controllers) are often configured so that no GERAN neighbor cells are provided during a video call. This configuration might slow down the re-selection from 3G to 2G when there is total loss of 3G coverage.

In summary, yes, users of Operator A will conne